<a href="https://colab.research.google.com/github/vishesh711/Development-of-a-Long-Context-Transformer-within-RAG-for-Enhanced-Document-Understanding/blob/main/Development_of_a_Long_Context_Transformer_within_RAG_for_Enhanced_Document_Understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation Plan
1. Set Up the Development Environment
Install Necessary Libraries

In [1]:
# Create a virtual environment (optional but recommended)
!python -m venv rag_long_context_env
!source rag_long_context_env/bin/activate  # On Windows use `rag_long_context_env\Scripts\activate`

# Upgrade pip
!pip install --upgrade pip

# Install required libraries
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117  # Adjust CUDA version as needed
!pip install transformers faiss-cpu datasets
!pip install --upgrade transformers


The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/rag_long_context_env/bin/python3

/bin/bash: line 1: rag_long_context_env/bin/activate: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing i

In [2]:
import torch
import transformers
import faiss

print(f"Torch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"FAISS version: {faiss.__version__}")


Torch version: 2.5.0+cpu
Transformers version: 4.45.2
FAISS version: 1.9.0


2. Choose a Base Long-Context Transformer Model
We will use Longformer as it is designed for long sequences and has efficient attention mechanisms.

In [3]:
from transformers import RagSequenceForGeneration

class RagLEDModel(RagSequenceForGeneration):
    def __init__(self, config, generator, retriever):
        super().__init__(config, generator=generator, retriever=retriever)
        # No need to set self.generator or self.retriever directly


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [5]:
# # Assuming you have already initialized the retriever and generator (LED model)
# # retriever: initialized earlier
# # model: your LEDForConditionalGeneration model

# # Create a new configuration for our RAG model
# from transformers import RagConfig

# config = RagConfig.from_pretrained('facebook/rag-token-nq')
# config.generator = model.config  # Update generator config to match LED

# # Initialize the custom RAG model
# rag_model = RagLEDModel(config=config, generator=model, retriever=retriever)


3. Update the RAG Integration
Since we're now using LED, which is designed for long-text summarization, we'll integrate it accordingly.

Define the Custom RAG Model with LED

In [6]:
from transformers import (
    LEDTokenizer,
    LEDForConditionalGeneration,
    RagRetriever,
    RagSequenceForGeneration,
    RagConfig,
)

# Load tokenizer and model
tokenizer = LEDTokenizer.from_pretrained('allenai/led-base-16384')
model = LEDForConditionalGeneration.from_pretrained('allenai/led-base-16384')

# Initialize the retriever
retriever = RagRetriever.from_pretrained(
    'facebook/rag-token-nq',
    index_name='exact',
    use_dummy_dataset=True  # Replace with actual dataset when available
)

# Define the custom RAG model with LED
class RagLEDModel(RagSequenceForGeneration):
    def __init__(self, config, generator, retriever):
        super().__init__(config, generator=generator, retriever=retriever)

# Create a new configuration for our RAG model
config = RagConfig.from_pretrained('facebook/rag-token-nq')
config.generator = model.config  # Update generator config to match LED

# Initialize the custom RAG model
rag_model = RagLEDModel(config=config, generator=model, retriever=retriever)


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/40.8M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


4. Prepare the Dataset
Use a dataset suitable for summarization of long documents, such as Multi-News or arXiv papers.

In [7]:
from datasets import load_dataset

# Load the Multi-News dataset
dataset = load_dataset('multi_news')


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

train.tgt:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

...

test.src.cleaned:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

5. Preprocess the Data
Adjust the maximum input length to accommodate the LED model's capabilities.

In [8]:
def preprocess_function(examples):
    inputs = examples['document']
    model_inputs = tokenizer(
        inputs,
        max_length=16384,  # LED can handle sequences up to 16,384 tokens
        truncation=True,
        padding='max_length'
    )
    labels = tokenizer(
        examples['summary'],
        max_length=512,
        truncation=True,
        padding='max_length'
    )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)


Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

6. Adjust the Attention Mask
LED uses a special attention mechanism that requires setting global attention on the first token of each batch.

In [1]:
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(example['input_ids']) for example in batch])
    attention_mask = torch.stack([torch.tensor(example['attention_mask']) for example in batch])
    labels = torch.stack([torch.tensor(example['labels']) for example in batch])

    # Set global attention on the first token
    global_attention_mask = torch.zeros_like(input_ids)
    global_attention_mask[:, 0] = 1

    batch = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'global_attention_mask': global_attention_mask
    }
    return batch


7. Update Training Arguments
Due to the large sequence length, you'll need to adjust batch sizes and possibly use gradient accumulation to fit the data into GPU memory.

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    eval_steps=500,
    per_device_train_batch_size=1,  # Start with batch size of 1
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,  # Accumulate gradients over 16 steps
    num_train_epochs=1,
    logging_steps=100,
    save_steps=1000,
    learning_rate=1e-4,
    fp16=True,  # Enable mixed precision training if supported
    dataloader_num_workers=4,
)


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


8. Initialize the Trainer with Updated Data Collator

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=rag_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=collate_fn,
)


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


9. Training the Model
Start training and monitor the GPU memory usage. You may need to adjust gradient_accumulation_steps and batch sizes based on your hardware capabilities.

In [ ]:
trainer.train()


10. Generate Summaries with the Trained Model
Define a function to generate summaries using the trained model.

In [2]:
def generate_summary(document):
    # Tokenize input
    inputs = tokenizer(
        document,
        return_tensors='pt',
        max_length=16384,
        truncation=True,
        padding='max_length'
    )

    # Set global attention mask
    global_attention_mask = torch.zeros_like(inputs['input_ids'])
    global_attention_mask[:, 0] = 1

    # Generate summary
    summary_ids = model.generate(
        input_ids=inputs['input_ids'].to(model.device),
        attention_mask=inputs['attention_mask'].to(model.device),
        global_attention_mask=global_attention_mask.to(model.device),
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


11. Testing the Updated Model

In [ ]:
test_document = dataset['test'][0]['document']
generated_summary = generate_summary(test_document)
print("Generated Summary:")
print(generated_summary)


12. Handling Multiple Documents
If you need to process multiple documents simultaneously (multi-document summarization), you can concatenate the documents and proceed similarly.

In [3]:
def generate_multi_doc_summary(documents):
    # Concatenate documents
    combined_document = ' '.join(documents)

    # Tokenize input
    inputs = tokenizer(
        combined_document,
        return_tensors='pt',
        max_length=16384,
        truncation=True,
        padding='max_length'
    )

    # Set global attention mask
    global_attention_mask = torch.zeros_like(inputs['input_ids'])
    global_attention_mask[:, 0] = 1

    # Generate summary
    summary_ids = model.generate(
        input_ids=inputs['input_ids'].to(model.device),
        attention_mask=inputs['attention_mask'].to(model.device),
        global_attention_mask=global_attention_mask.to(model.device),
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


13. Evaluate the Model
Use evaluation metrics suitable for summarization, such as ROUGE.

In [4]:
from datasets import load_metric
import nltk

nltk.download('punkt')
rouge = load_metric('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ['\n'.join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return result

# Update the trainer
trainer.compute_metrics = compute_metrics

# Run evaluation
eval_results = trainer.evaluate()
print(eval_results)


ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.10/dist-packages/datasets/__init__.py)

14. Troubleshooting Tips
GPU Memory Issues: If you encounter out-of-memory errors, try reducing per_device_train_batch_size or increasing gradient_accumulation_steps.
Sequence Length Limitations: Ensure that the maximum sequence length (max_length) does not exceed 16,384 tokens for the LED model.
Version Compatibility: Verify that you are using transformers version 4.2.0 or higher.

15. Additional Considerations
Efficient Training: Use techniques like gradient checkpointing to reduce memory consumption.

In [ ]:
model.gradient_checkpointing_enable()


FP16 Training: Mixed precision training can speed up training and reduce memory usage.

Distributed Training: If you have access to multiple GPUs, consider using distributed training strategies provided by transformers.